In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
# use process bar tool
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-final-dataset/dataset/test/adults/20.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/6.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/76.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/71.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/5.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/8.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/84.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/85.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/67.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/82.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/30.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/10.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/0.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/62.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/61.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/73.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/60.jpg
/kaggle/input/ml-f

In [2]:
torch.__version__

'2.1.2+cpu'